In [ ]:
from app.Labeler import Labeler
import pandas as pd
from app.device_namespace import SNESORS_CSV
from app.labels_namespace import ORIENTATION, OTHER
from app.utils import load_data_for_device
import copy
from app.video_utils import draw_plot_plt, cut_video, merge_videos
from app.DataTransformer import DataTransformer


In [ ]:
current_date = "2024-02-25"
labeler = Labeler(date=current_date)
df_dict = load_data_for_device(device_names=list(labeler.folders_dict.keys()),current_date=current_date)
honor_data = copy.deepcopy(df_dict['HONOR_8X'])

In [ ]:
honor_data.index

In [ ]:
import math

def scale_angle(angle):
    scaled_angle = (angle + 2 * math.pi) % (2 * math.pi)
    return scaled_angle

def data_transformation(dataframe : pd.DataFrame, smoothing = True, scaling = True):
    """applay transformations such as scaling angels and smoothing (for now only to the orientation data)"""
    large_dict_dict = {}
    for key in list(dataframe.keys()):
        if key == 'time':
            continue
        elif key == 'Orientation':
            temp = pd.DataFrame(dataframe['Orientation'].tolist(), columns=ORIENTATION)
            #scaling
            if scaling:
                temp = temp.applymap(scale_angle)

            #normalizacja
            #temp = temp.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
            if smoothing:
                temp = temp.rolling(window=3, min_periods=1).mean()

        else:
            temp = pd.DataFrame(dataframe[key].tolist(), columns=OTHER)


        temp['time'] = honor_data['time']

        large_dict_dict[key] = temp
    return large_dict_dict

#smoothed and scaled data
honor_smooth_scaled = data_transformation(honor_data)
final_smooth_scaled = DataTransformer.prepare_final_df(device_dataframes = honor_smooth_scaled)


In [ ]:
#data need to be 2 sec before the video (12:44:01 - camera time and 12:44:03 - rel time
# so if we want to take a movie from 12:46:48 - 12:48:02 data need to be from 12:46:50 - 12:48:04
draw_plot_plt(df = final_smooth_scaled, start= "12:46:50", end="12:48:04", output= "sensors_data", col = 'Orientation')


In [ ]:
#function that will transfer minute to sec like 2:47
def time_to_sec(time):
    time = time.split(":")
    return str(int(time[0])*60 + int(time[1]))

In [ ]:
time_to_sec("2:47")

In [ ]:
#2:47 - 4:01
cut_video(input_path= "2024_0225_124401_005.MP4", output_path="skiers_data.mp4" start = time_to_sec("2:47"), end = time_to_sec("4:01"))